# Getting Started

In this notebook, we are going to explore some of the functionalities that Muse API provides and build some example chatbot applications with a focus on natural conversation.

For a quick intro on machine learning approaches to building chatbots, [this](http://www.wildml.com/2016/04/deep-learning-for-chatbots-part-1-introduction/) is a good primer.

## Retrieval based model

We will start with an example of how we can build a retrieval based chatbot using Muse API.

Muse API provides access to a number of data sources from which you can retrieve potential responses for a given user input. The url for this function is `/api/chat/retrieval/`. Let's `import` a wrapper function that sends a request and returns the output.

In [1]:
from muse import get_candidate_responses

The `get_candidate_responses` function takes two arguments, `sent`, and `source`. The `sent` argument is the sentence to which we want to retrieve candidate responses for, and the `source` argument specifies from which data source the responses come. At the moment, there are three data sources, which you can access by setting the `source` argument to one of `[1,2,3]`. Let's retrieve potential responses for the sentence **"What did you last weekend?"** from data source `3` and print the similarity score and the response text of the first five candidates.

In [2]:
candidate_responses = get_candidate_responses("what did you last weekend?",source=3)
for candidate in candidate_responses[:5]:
    print(candidate['score'], candidate['text'])

(0.7690440580433996, u'I went to a festival was such a good time')
(0.5524923548780294, u'I watched the new Star Wars Movie Rogue One!')
(0.5524923548780294, u'Oh, I just sat home on the couch watching tv.')
(0.5336215919403026, u'Oh I just stayed in bed and watched Aladdin.')
(0.5212465751011512, u'I went to go see a concert at The National.')


We can simply return the candidate response with the highest score (the candidates are ordered in the list by descending similarity score), and build a super simple retrieval based chatbot

In [23]:
while True:
    user_text = raw_input("User> ")
    if user_text.lower().strip() in ['quit','stop','bye']:
        print("Bot> Alright, see you later.")
        break
    candidates = get_candidate_responses(user_text,source=3)
    bot_response = candidates[0]['text']
    print("Bot> %s"%bot_response)

User> hey
Bot> Hey man
User> what did you do last weekend?
Bot> I went to a festival was such a good time
User> what festival did you go to?
Bot> I went to the cineplex 8 and the fairview mall.
User> do they show movies or something?
Bot> I like war movies?
User> i don't know.
Bot> That sounds like a live version? Is it?
User> maybe
Bot> Well who knows
User> you should know because you went to the festival
Bot> Oh yes!  I went to see the GoGos perform at a small college in Illinois. I think I was about 18 at the time?
User> who are the GoGos?
Bot> Well, let's see, Reggie Miller, Steve Nash, Larry Bird, to name a few, and that's shooters who also played defense.
User> so it's a basketball team?
Bot> I like the New York Rockets?
User> maybe?
Bot> Just listen to them.  Nothing THAT unique
User> bye
Bot> Alright, see you later.


## Machine Comprehension

Now, let's make a bot that responds to
1. statements with a simple "Okay.", and 
2. questions using the machine comprehension model on previous user statements.

So the flow we have in mind is the following. The user utters several statements, and when the user asks a question at the end, the chatbot tries to answer the question based on what the user said. 

First, let's import the wrapper function that does machine comprehension.


In [3]:
from muse import comprehend

The `comprehend` function takes two arguments, `passage`, and `question`, and returns the answer to the question about the passage. Let's try it out quickly.

In [28]:
passage = "My favorite video game is Skyrim."
question = "What video game do I like to play?"
answer = comprehend(passage,question)
print(answer)

Skyrim


Second, we need a helper function that will determine whether a given user utterance is a statement or a question. So let's make a super simple function that checks whether the first word of the user utterance is a usual first word in questions, like 'why','what','when',etc.

In [4]:
def is_question(utterance):
    question_start_tokens = ['what','how','where','who','why','which','when']
    first_token = utterance.split()[0].lower()
    if first_token in question_start_tokens:
        return True
    else:
        return False
    

In [25]:
user_utts_hist = []
while True:
    user_text = raw_input("User> ")
    if user_text.lower().strip() in ['quit','stop','bye']:
        print("Bot> Alright, see you later.")
        break
    
    if is_question(user_text):
        answer = comprehend(' '.join(user_utts_hist),user_text)
        print("Bot> %s"%answer)
    else:
        user_utts_hist.append(user_text)
        print("Bot> Okay.")

User> my parents live in Busan.
Bot> Okay.
User> my parents are in their 50s.
Bot> Okay.
User> my dad is older than my mom by one year.
Bot> Okay.
User> how much older is my dad than my mom?
Bot> by one year
User> where do my parents live?
Bot> Busan
User> how old are my parents?
Bot> 50s
User> my father is a music teacher.
Bot> Okay.
User> my mother is a family therapist.
Bot> Okay.
User> what does my father do?
Bot> a music teacher
User> what does my mother do?
Bot> a family therapist
User> my brother is in the military.
Bot> Okay.
User> i am sad because my brother is in the military.
Bot> Okay.
User> where is my brother?
Bot> in the military
User> why am i sad?
Bot> my brother is in the military
User> bye
Bot> Alright, see you later.


## Coreference Resolution

Let's improve the above code, so that we don't have to keep saying my parents, my father, my mother, my brother, etc. everytime and instead say they, he, she, etc. For this we are going to use the coreference resolution feature provided by Muse API. The wrapper function is `coreference_resolution`. It takes two arguments, `context`,and `utterance`. Both are strings, and it will resolve any occurances of "reference words" in the `utterance` with appropriate noun chunks from `utterance`.

In [26]:
from muse import coreference_resolution

Let's try it out quickly.

In [31]:
context = "my mom is a family therapist. my dad is a music teacher."
utterance = "sometimes she gets mad at him."
resolved = coreference_resolution(context,utterance)
print(resolved)

sometimes my mom gets mad at my dad.


So far, so good. Let's try and put it into our comprehension bot. We should set a limit on how far back into the past utterances to search for noun chunks to replace the pronouns in a given utterance. Let's set this number to 3.

In [32]:
COREF_UTTS_WINDOW = 3
user_utts_hist = []
while True:
    user_text = raw_input("User> ")
    if user_text.lower().strip() in ['quit','stop','bye']:
        print("Bot> Alright, see you later.")
        break
    
    coref_context = ' '.join(user_utts_hist[-COREF_UTTS_WINDOW:])
    resolved_user_utts = coreference_resolution(coref_context,user_text)
    if is_question(user_text):
        answer = comprehend(' '.join(user_utts_hist),resolved_user_utts)
        print("Bot> %s"%answer)
    else:
        user_utts_hist.append(resolved_user_utts)
        print("Bot> Okay.")

User> my dad is a music teacher.
Bot> Okay.
User> my mother is a family therapist.
Bot> Okay.
User> sometimes she gets mad at him.
Bot> Okay.
User> who does she get mad at?
Bot> my dad
User> what does he do?
Bot> a music teacher
User> bye
Bot> Alright, see you later.
